In [3]:
import pandas as pd
import numpy as np

In [1]:
cd D:\Darvin\changepond_task

D:\Darvin\changepond_task


In [4]:
df = pd.read_csv('rec.csv',names=['user_id', 'movie_id', 'rating', 'timestamp'])
df

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [6]:
from sklearn.model_selection import train_test_split
n_users = df.user_id.unique().shape[0] 
n_movies = df.movie_id.unique().shape[0]
train_data, test_data = train_test_split(df, test_size=0.25)

In [9]:
train_data_matrix = np.zeros((n_users, n_movies))
for line in train_data.itertuples():
    #[user_id index, movie_id index] = given rating.
    train_data_matrix[line[1]-1, line[2]-1] = line[3] 
train_data_matrix

array([[0., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [8]:
test_data_matrix = np.zeros((n_users, n_movies))
for line in test_data.itertuples():
    #[user_id index, movie_id index] = given rating.
    test_data_matrix[line[1]-1, line[2]-1] = line[3]
test_data_matrix

array([[5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
from sklearn.metrics import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
movie_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')
mean_user_rating = train_data_matrix.mean(axis=1)[:, np.newaxis] 
ratings_diff = (train_data_matrix - mean_user_rating) 
user_pred = mean_user_rating + user_similarity.dot(ratings_diff) / np.array([np.abs(user_similarity).sum(axis=1)]).T
user_pred

array([[ 1.56015856,  0.5689003 ,  0.47992796, ...,  0.29055062,
         0.29050875,  0.29036226],
       [ 1.24678318,  0.27886908,  0.14926836, ..., -0.06745542,
        -0.06643333, -0.06622371],
       [ 1.28726068,  0.24720734,  0.12595168, ..., -0.10050244,
        -0.09881779, -0.09869168],
       ...,
       [ 1.16704509,  0.21654167,  0.09626154, ..., -0.11727574,
        -0.1164318 , -0.11619599],
       [ 1.331968  ,  0.31610678,  0.21345852, ..., -0.00576399,
        -0.00473797, -0.00477726],
       [ 1.39631718,  0.39286341,  0.31202979, ...,  0.11823914,
         0.11822057,  0.11820885]])

In [11]:
movie_pred = train_data_matrix.dot(movie_similarity) / np.array([np.abs(movie_similarity).sum(axis=1)])
movie_pred

array([[0.37053138, 0.379262  , 0.39060711, ..., 0.44174827, 0.43149799,
        0.42824734],
       [0.08044422, 0.09528493, 0.09175119, ..., 0.09807149, 0.09806991,
        0.09875874],
       [0.06402058, 0.06879222, 0.0660028 , ..., 0.06466385, 0.06752276,
        0.06820281],
       ...,
       [0.03383866, 0.042445  , 0.04129036, ..., 0.04748544, 0.04655943,
        0.04736324],
       [0.1318234 , 0.14061309, 0.14699641, ..., 0.15271805, 0.15281308,
        0.15257287],
       [0.22080865, 0.21502632, 0.23540219, ..., 0.27103112, 0.26189964,
        0.26173919]])

In [12]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(pred, test):
    pred = pred[test.nonzero()].flatten() 
    test = test[test.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, test))

In [13]:
rmse(user_pred, test_data_matrix)

3.1307035770996774

In [21]:
rmse(movie_pred, test_data_matrix)

3.4647957787706685